In [1]:
# Import Libraries
import tweepy as tw
import configparser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import re
import nltk
import datetime
from datetime import datetime as datetime_price
import pytz
from dateutil.relativedelta import relativedelta
import time

# Authentication for API

In [2]:
keys = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\TW API Keys\apikeys.csv")

my_api_key = keys["Keys"][0]
my_api_secret = keys["Keys"][1]

my_access_token = keys["Keys"][2]
my_access_token_secret = keys["Keys"][3]

In [3]:
# your Twitter API key ans access token
api_key = my_api_key
api_secret = my_api_secret

access_token = my_access_token
access_token_secret = my_access_token_secret

In [4]:
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
auth.set_access_token(access_token, access_token_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

# Insert the List of Users

In [6]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding= 'unicode_escape')
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk_doge,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,SHIB,shiba-inu,Elon Musk_shiba,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
9,SOL,solana,Solana,Solana,solana,company,there is no account of CEO or sth


# Taking Tweets of List of Users

user = "el33th4xor"
limit = 300

In [7]:
#datetime format -> (year, month, day, hour, minute, second)
endDate = datetime.datetime(2022, 9, 27, 23, 59, 59)
startDate = endDate-relativedelta(days = 50)
#endDate = datetime.datetime.now()
#startDate = endDate - relativedelta(days = 1) #days, months, years can be used to calculate

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate)

In [8]:
columns = ["Tweet_id", "Name", "Screen_name", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav", "Related_coin"]

--old version of taking tweets
#cursor can be used for just one time, be careful about that, save and use use data frame/s for next steps
tweets = tw.Cursor(api.user_timeline, 
                   screen_name = user, count = 200, 
                   tweet_mode = "extended").items(limit)

In [ ]:
def take_tweets(sc_name):
    tweets = []
    tmpTweets = api.user_timeline(screen_name = sc_name)
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
        
    while (tmpTweets[-1].created_at > startDate):
        tmpTweets = api.user_timeline(screen_name = sc_name, max_id = tmpTweets[-1].id)
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets.append(tweet)
    return tweets

In [16]:
def take_tweets_2(sc_name):
    tweets = []
    
    
    while True:
        try:
            tmpTweets = api.user_timeline(screen_name = sc_name)
            for tweet in tmpTweets:
                if tweet.created_at < endDate and tweet.created_at > startDate:
                    tweets.append(tweet)
            break
        except tw.errors.TweepyException as e:
            print(e)
            time.sleep(60*15)
            continue
    
    while True:
        try:
            while(len(tmpTweets) > 0) and (tmpTweets[-1].created_at > startDate):
                tmpTweets = api.user_timeline(screen_name = sc_name, max_id = tmpTweets[-1].id)
                for tweet in tmpTweets:
                    if tweet.created_at < endDate and tweet.created_at > startDate:
                        tweets.append(tweet)
            break
        except tw.errors.TweepyException as e:
            print(e)
            time.sleep(60*15)
            continue
        
    return tweets

In [10]:
def categorize_tweets(tweets):
    
    data = []
    term_rt = "RT"
    for tweet in tweets:
    
        #for consecutive tweets, next one is shown primaryly in data frame, I can check with using hours and minutes if necessary
        date = tweet.created_at
        #date = date.strftime('%Y-%m-%d')
    
        #retweet and favorite counts
        number_of_retweet = tweet.retweet_count
        number_of_favorite = tweet.favorite_count
    
    
        #it's gonna be direct tweet if all the checks below fail
        category_specified = "direct"

        #retweet check
        if term_rt in tweet.text.split():

            category_specified = "retweet"

        #quote(rt with comment) check
        elif tweet.is_quote_status is True:
            category_specified = "quote"

        #reply check
        elif tweet.in_reply_to_status_id is not None:
            category_specified = "reply"

        data.append([tweet.id, tweet.user.name, tweet.user.screen_name, tweet.text, date, category_specified, number_of_retweet, number_of_favorite, ""])
    
    df_met = pd.DataFrame(data, columns = columns)
    return df_met

In [11]:
def tweetCollector(user_df):
    df = pd.DataFrame(columns = columns)

    index_counter_1 = 0;
    for x in user_df.Screen_name:
    
        taken_tweets = take_tweets(x)
        df_met_2 = categorize_tweets(taken_tweets)

        df_met_2["Related_coin"] = user_df["ID"][index_counter_1]
        index_counter_1 += 1

        df = df.append(df_met_2)

    df = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]
    
    return df

In [17]:
df = tweetCollector(user_df)

Rate limit reached. Sleeping for: 537


Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


Rate limit reached. Sleeping for: 508


Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


Rate limit reached. Sleeping for: 558


...

Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


Rate limit reached. Sleeping for: 552


Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


Rate limit reached. Sleeping for: 554


Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [1]:
"""
df = pd.DataFrame(columns = columns)

index_counter_1 = 0;
for x in user_df.Screen_name:
    
    taken_tweets = take_tweets(x)
    df_met_2 = categorize_tweets(taken_tweets)
    
    df_met_2["Related_coin"] = user_df["ID"][index_counter_1]
    index_counter_1 += 1
    
    df = df.append(df_met_2)

df = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]
"""

'\ndf = pd.DataFrame(columns = columns)\n\nindex_counter_1 = 0;\nfor x in user_df.Screen_name:\n    \n    taken_tweets = take_tweets(x)\n    df_met_2 = categorize_tweets(taken_tweets)\n    \n    df_met_2["Related_coin"] = user_df["ID"][index_counter_1]\n    index_counter_1 += 1\n    \n    df = df.append(df_met_2)\n\ndf = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]\n'

In [28]:
df

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav,Day,Month,Year
0,1573510876770050053,Brad Garlinghouse,bgarlinghouse,ripple,RT @ss_friedman: The veto of AB 2269 gives CA ...,2022-09-24 03:13:22,retweet,542,0,24,9,2022
1,1573290507715084288,Brad Garlinghouse,bgarlinghouse,ripple,@CGasparino Charlie G! FWIW I appreciate you ...,2022-09-23 12:37:42,reply,135,1346,23,9,2022
2,1572698636013764609,Brad Garlinghouse,bgarlinghouse,ripple,"@JoelKatz David, I'm adding this to our next 1...",2022-09-21 21:25:48,reply,80,2278,21,9,2022
3,1572319964027035648,Brad Garlinghouse,bgarlinghouse,ripple,RT @POLITICOLive: TUNE IN NOW to “Writing the ...,2022-09-20 20:21:06,retweet,393,0,20,9,2022
4,1571922558404145158,Brad Garlinghouse,bgarlinghouse,ripple,An absolute pleasure to welcome you to our off...,2022-09-19 18:01:57,quote,1316,6398,19,9,2022
5,1571281889008263168,Brad Garlinghouse,bgarlinghouse,ripple,Today's filings make it clear the SEC isn’t in...,2022-09-17 23:36:10,quote,2212,7157,17,9,2022
6,1571281729406525440,Brad Garlinghouse,bgarlinghouse,ripple,RT @s_alderoty: My hot take - after two years ...,2022-09-17 23:35:31,retweet,1181,0,17,9,2022
7,1570334032587620354,Brad Garlinghouse,bgarlinghouse,ripple,Truly an incredible technological feat – congr...,2022-09-15 08:49:43,direct,666,5671,15,9,2022
8,1569819675412041729,Brad Garlinghouse,bgarlinghouse,ripple,Smellkis and Garlichouse on the same stage nex...,2022-09-13 22:45:51,direct,387,2159,13,9,2022
9,1567540557114753025,Brad Garlinghouse,bgarlinghouse,ripple,RT @Ripple: We are proud to be included on @Fo...,2022-09-07 15:49:26,retweet,488,0,7,9,2022


In [19]:
#removing duplications, reseting index
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)

In [20]:
#seperating dat-month-year
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Year'] = pd.DatetimeIndex(df['Date']).year

In [21]:
df.shape

(3178, 12)

In [22]:
#null check --> if it is "False", we all ok
df.isna().any().any()

False

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3178 entries, 0 to 3177
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tweet_id      3178 non-null   object
 1   Name          3178 non-null   object
 2   Screen_name   3178 non-null   object
 3   Related_coin  3178 non-null   object
 4   Tweet         3178 non-null   object
 5   Date          3178 non-null   object
 6   Category      3178 non-null   object
 7   N_of_RT       3178 non-null   object
 8   N_of_fav      3178 non-null   object
 9   Day           3178 non-null   int64 
 10  Month         3178 non-null   int64 
 11  Year          3178 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 298.1+ KB


In [24]:
#duplication check
df[df.duplicated(keep=False)].shape[0]

0

In [26]:
df['Date'] = df['Date'].dt.tz_localize(None)

In [27]:
pd.set_option("max_rows", None)
df.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Tweet Data\tweets_4.xlsx")
df.head()

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav,Day,Month,Year
0,1573510876770050053,Brad Garlinghouse,bgarlinghouse,ripple,RT @ss_friedman: The veto of AB 2269 gives CA ...,2022-09-24 03:13:22,retweet,542,0,24,9,2022
1,1573290507715084288,Brad Garlinghouse,bgarlinghouse,ripple,@CGasparino Charlie G! FWIW I appreciate you ...,2022-09-23 12:37:42,reply,135,1346,23,9,2022
2,1572698636013764609,Brad Garlinghouse,bgarlinghouse,ripple,"@JoelKatz David, I'm adding this to our next 1...",2022-09-21 21:25:48,reply,80,2278,21,9,2022
3,1572319964027035648,Brad Garlinghouse,bgarlinghouse,ripple,RT @POLITICOLive: TUNE IN NOW to “Writing the ...,2022-09-20 20:21:06,retweet,393,0,20,9,2022
4,1571922558404145158,Brad Garlinghouse,bgarlinghouse,ripple,An absolute pleasure to welcome you to our off...,2022-09-19 18:01:57,quote,1316,6398,19,9,2022


In [37]:
#seperation according to categories 
retweet_df = df[df["Category"] == "retweet"]
retweet_df.reset_index(inplace = True, drop=True)
quote_df = df[df["Category"] == "quote"]
quote_df.reset_index(inplace = True, drop=True)
reply_df = df[df["Category"] == "reply"]
reply_df.reset_index(inplace = True, drop=True)
direct_df = df[df["Category"] == "direct"]
direct_df.reset_index(inplace = True, drop=True)
retweet_df.head()

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav,Day,Month,Year
0,1609312409491066882,vitalik.eth,VitalikButerin,ethereum,RT @VitalikButerin: @IAPonomarenko Happy new y...,2022-12-31,retweet,545,0,31,12,2022
1,1609050268632772611,Charles Hoskinson,IOHK_Charles,cardano,RT @ikigai_myagi: @IOHK_Charles Took this pict...,2022-12-31,retweet,11,0,31,12,2022
2,1609207496434159621,Sandeep | Polygon 💜 Top 3 by impact,sandeepnailwal,matic-network,RT @eaglelex_eth: 11/The work at @0xPolygon 💜 ...,2022-12-31,retweet,3,0,31,12,2022
3,1609207483825889280,Sandeep | Polygon 💜 Top 3 by impact,sandeepnailwal,matic-network,RT @eaglelex_eth: 1/The future of #DeFi \n\nSo...,2022-12-31,retweet,13,0,31,12,2022
4,1609207102320377858,Sandeep | Polygon 💜 Top 3 by impact,sandeepnailwal,matic-network,RT @thematicman: 🎆 10 POLYGON NFT PROJECTS SET...,2022-12-31,retweet,65,0,31,12,2022


# Taking User Object

In [ ]:
user_try = api.get_user(screen_name = "el33th4xor")
user_try.name

# Follower - Followed

check this part of code later??????????????? cursor object is not itterable?

In [ ]:
user = "el33th4xor"
followers = []
get_followers = api.get_followers(screen_name = user)
#get_followers = tw.Cursor(api.get_followers, id = user)

for user in get_followers:
    followers.append(user)

In [ ]:
len(followers)

In [ ]:
columns_follower = ["User_id", "Name", "Screen_name", "Verification_st", "Followers_count", "Followed_count"]
data_followers = []

In [ ]:
for user in followers:
    data_followers.append([user.id, user.name, user.screen_name, user.verified, user.followers_count, user.friends_count])

In [ ]:
df_follower = pd.DataFrame(data_followers, columns = columns_follower)
df_follower

In [ ]:
df_follower.drop_duplicates(inplace = True)
df_follower.reset_index(inplace = True, drop = True)

# Statistics and Visualization

Graph should be addepted to new version of df(with lots of users)

In [ ]:
df.corr()

In [ ]:
plot1 = retweet_df.groupby("Day")["Tweet_id"].count()
plot2 = quote_df.groupby("Day")["Tweet_id"].count()
plot3 = reply_df.groupby("Day")["Tweet_id"].count()
plot4 = direct_df.groupby("Day")["Tweet_id"].count()
plt.figure(figsize = (14,7))
ax= plt.subplot()
ax.bar(plot1.index-0.3, plot1.values, width = 0.2, color = "pink", align = "center", label = "Retweet")
ax.bar(plot2.index-0.1, plot2.values, width = 0.2, color = "yellow", align = "center", label = "Quote")
ax.bar(plot3.index+0.1, plot3.values, width = 0.2, color = "purple", align = "center", label = "Reply")
ax.bar(plot4.index+0.3, plot4.values, width = 0.2, color = "grey", align = "center", label = "Direct")
plt.title("Number of Tweets According to Category per Day")
plt.ylabel("Number of Tweets")
plt.xlabel("Days")
#they are not sorted from 1 to 30 or 31, last they is today and first day is one month earlier. There is need for a solution also.
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]) #find a solution for changes of number of days in a month
plt.legend()
plt.show()